In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#Jets adding
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [13]:
#import combined data from csv
data = pd.read_csv("C:\\Users\\grovenbm\\Documents\\oms_analytics\\CSE 6242\\Project\\Tests\\philly_final_dataset_Reduced.csv")



In [22]:
y = data['sale_price']
data_Features = data.drop(columns=['sale_price','Est TOM in Days'])
data_hot = data_Features
data_hot = pd.get_dummies(data_hot, drop_first=True)


In [23]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(data10_hot, y, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(data_hot, y, test_size=0.3, random_state=1)

# Standardize full feature set for Elastic Net variable selection
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)
X_test_scaled_full = scaler_full.transform(X_test)

In [24]:
# Variable selection with Elastic Net
elasticNet = ElasticNetCV(l1_ratio=0.5, cv=5)
elasticNet.fit(X_train_scaled_full, y_train)

# Get the selected features
coefficients = elasticNet.coef_
selected_features = data_hot.columns[coefficients != 0]
print("Selected features:")
print(selected_features)

Selected features:
Index(['zip_code', 'central_air', 'garage_spaces', 'number_of_bedrooms',
       'number_of_bathrooms', 'number_stories', 'total_livable_area',
       'total_area', 'year_built', 'has_basements', 'active_inventory',
       'new_listings', 'pending_sales', 'homes_sold', 'sale_to_list_ratio',
       'percent_above_list', 'avg_days_on_market', 'avg_list_price_per_sqft',
       'avg_listed_price', 'avg_price_per_sqft', 'avg_price_sold'],
      dtype='object')


In [25]:
print("--------------------------------------End of Variable Selection--------------------------------------")

--------------------------------------End of Variable Selection--------------------------------------


In [26]:
#Instiantiate model
LMmodel = LinearRegression()

#fit model to training data. Using only selected features
X_train_selected = X_train[selected_features]

LMmodel.fit(X_train_selected, y_train)

#predict on test data
x_test_selected = X_test[selected_features]
y_pred = LMmodel.predict(x_test_selected)

print("Predictions:")
print(y_pred)

#calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred)
print(R2)

#Access coefficients
coefficients = LMmodel.coef_
print("Coefficients:")
print(coefficients)

#Access intercept
intercept = LMmodel.intercept_
print("Intercept:")
print(intercept)

Predictions:
[-536867.07825505   74352.83834669 1013906.94758709 ... 1887590.41706514
  114726.50446234  964439.23253535]
RMSE:
959831.7116652863
R2:
0.2727150065364611
Coefficients:
[-2.54615666e+03  1.78051550e+05  2.22946209e+05  3.51178029e+04
  9.00308543e+04  6.43461597e+05  1.27498317e+01  1.16779265e-02
 -1.36256168e+02 -3.91394065e+05  9.02889870e+02 -4.97894032e+03
  1.03990159e+04 -5.15106133e+03 -8.17391891e+05 -7.58625667e+04
 -1.22987216e+03  4.12535581e+02  3.08817598e-01  7.33931129e+02
 -6.71925879e-01]
Intercept:
48756421.45380745


In [27]:
#Instantiating Random Forest Model
RFmodel = RandomForestRegressor(n_estimators=50, random_state=1,n_jobs=-1 ,verbose=1)

#RF model fit
RFmodel.fit(X_train, y_train)

#Sanity check
print("Progress...")

#Predicting on test data
y_pred_RF = RFmodel.predict(X_test)
print("Predictions:")
print(y_pred_RF)

# Calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_RF))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred_RF)
print(R2)

# Get feature names
feature_names = X_train.columns

# Access feature importances (since Random Forest models provide feature importance)
print("Feature Importances:")
feature_importance = pd.Series(RFmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s


Progress...
Predictions:
[ 212970.          133028.16357143  540709.9        ... 3009170.16
  171568.88        460658.62      ]
RMSE:
566252.6303970212
R2:
0.7468750803573763
Feature Importances:
[9.27631290e-03 9.80639952e-03 7.33431192e-03 1.56383374e-02
 2.14114143e-02 2.84773444e-01 4.51487731e-01 5.07960405e-02
 3.05241380e-02 9.59154909e-05 7.20936429e-03 1.59179289e-02
 1.01517165e-02 1.56003956e-02 1.47300594e-02 1.07202496e-02
 9.74269540e-03 0.00000000e+00 6.86338955e-03 8.71805766e-03
 6.86706236e-03 8.40872584e-03 3.92630965e-03]


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [28]:
# Instantiate Gradient Boosting model

'''
GBmodel = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.15,
    max_depth=4,
    random_state=1,
    verbose=1
)
'''
GBmodel = GradientBoostingRegressor(
    subsample= 1.0, n_estimators= 500,
    min_samples_split= 2, min_samples_leaf= 1,
    max_features= None, max_depth= 4, learning_rate= 0.15
)

# Fit the model on untransformed target
GBmodel.fit(X_train, y_train)

# Predict on test data
y_pred_GB = GBmodel.predict(X_test)

# Evaluate performance
RMSE_GB = np.sqrt(mean_squared_error(y_test, y_pred_GB))
R2_GB = r2_score(y_test, y_pred_GB)

print("Predictions:")
print(y_pred_GB)
print("RMSE (Gradient Boosting):", RMSE_GB)
print("R² (Gradient Boosting):", R2_GB)

# Get feature names
feature_names = X_train.columns

# Access feature importances
print("Feature Importances:")
feature_importance = pd.Series(GBmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

Predictions:
[ 186185.3796489   154549.64214761  503762.22204732 ... 4570769.83245907
  172904.81040553  532677.65280899]
RMSE (Gradient Boosting): 577112.8228512147
R² (Gradient Boosting): 0.7370725764255591
Feature Importances (Gradient Boosting):
[1.12684896e-02 6.55851414e-03 7.64903879e-03 7.19577463e-03
 4.86027070e-02 3.32697856e-01 3.83818375e-01 6.32154839e-02
 2.06151660e-02 4.62885110e-05 2.93880273e-03 5.77268052e-03
 3.85696862e-03 3.68439183e-03 9.26540202e-03 9.98097445e-03
 3.01570027e-02 0.00000000e+00 1.68408292e-03 3.22831284e-02
 4.47526621e-03 7.01341834e-03 7.22018792e-03]


In [29]:
# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiate base model
gbr = GradientBoostingRegressor(random_state=1)

# Set up randomized search
rand_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=50,               # Number of random combinations to try
    scoring='r2',            # Optimize for R² score
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,               # Use all available cores
    random_state=42
)

# Fit to training data
rand_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters Found:")
print(rand_search.best_params_)
print("Best Cross-Validated R² Score:")
print(rand_search.best_score_)

# Evaluate on test set
best_gbr = rand_search.best_estimator_
y_pred_best = best_gbr.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print("\nTest Set Evaluation")
print("RMSE:", rmse_best)
print("R²:", r2_best)

#with open("modelSale.pkl", "wb") as f:
    #import pickle
joblib.dump(best_gbr, 'modelSale.pkl')


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters Found:
{'subsample': 1.0, 'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 3, 'learning_rate': 0.1}
Best Cross-Validated R² Score:
0.5940243816784413

Test Set Evaluation
RMSE: 686769.4148392775
R²: 0.6276628851536792


['modelSale.pkl']